# With metadata to better data! (Meta)Data transfer from and to Coscine

The [Coscine research data platform](https://www.coscine.de) provides an API interface to transfer metadata annotated data to Coscine in automated processes. In the workshop, we will show in small-scale steps how to move data to Coscine using a JupyterNotebook (Python) and Coscine's personal authentication token, and how to specify the metadata using the application profile provided by the application. Prior knowledge of Python is desirable.

First things first, you need a Coscine project. You can use your own if you have access, or we will add you to this one: https://coscine.rwth-aachen.de/p/fdmwerkstatt/ 

Next, head to your [user profile](https://coscine.rwth-aachen.de/user/) and get your Access Token. Copy this into your config file under `token`.


Create a resource... 

Go to the resource setting and copy all relevant information into your config file. This includes resource name and ID and project name. 

Now let's load all dependencies and configurations into our jupyter notebook.

If you have any errors loading the packages, run the code below with the associated package name:

In [ ]:
pip install PACKAGE_NAME

Load the configuration:

We use the Coscine package to connect with Coscine REST API, which enables us to interact with our project and resource. 

For more information and other examples: [Coscine Python SDK](https://git.rwth-aachen.de/coscine/community-features/coscine-python-sdk)

Get the metadata form and take a look at it:

This form is a dictionary-like data structure, so you can interact with it like a python dictionary:

The error is because the field is a controlled vocabulary. Let's see what is allowed:

Let's deal with the date. It needs to be formatted as a datetime object:

Add whatever else is missing:

Now we upload the metadata and the data. We'll just use a dummy text file here called `myData.txt`.

<!-- If you were working with a nested directory structure or larger data, you'd want to use the S3 credentials to interact with the resource via s3 protocol. 

We can get these using the API: -->

Both web resources and S3 resources allow you to create directories. But web resources require metadata be provided.

This let's us make directories:

And upload files to a directory:

We cannot add metadata via S3, so we use the API to update the metadata:

Let's add metadata to the folder as well:

## Let's get fancier and extract metadata from a file

Let's try getting some metadata out of an image file. For this we use the pillow (PIL) module.

The data folder includes a dataset titled 3dsem which includes some JPEG and TIFF images

data source
Authors: Tafti, Ahmad P and Kirkpatrick, Andrew B and Holz, Jessica D and Owen, Heather A and Yu, Zeyun

DOI: 10.7910/DVN/HVBW0Q

License: CC-0

We can extend the base profile in Coscine to fit some of this metadata. Which we've already done! Yay!

Let's create a new resource within out project. We can do this via Coscine web interface.